In [86]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as plt

import time
import requests

from shapely.geometry import Point
from shapely.geometry import Polygon

%matplotlib inline

In [1]:
MAPZEN_KEY = 'mapzen-kLUbkiJ'

In [5]:
listings = gpd.read_file('Data/NYClisting.geojson')

In [7]:
len(listings)

40586

In [8]:
listings.crs

{'init': 'epsg:4326'}

In [11]:
# Get 🍺
beer = gpd.read_file('Data/yelp_beer.geojson')
beer.crs = {'init' :'epsg:4326'}

In [17]:
len(beer)

67

In [15]:
# Get ☕️
coffee = gpd.read_file('Data/coffee.geojson')
coffee.crs = {'init' :'epsg:4326'}

In [18]:
len(coffee)

375

## Merge

In [66]:
poly = r.json()['features'][0]['geometry']['coordinates']

In [90]:
def generate_walksheds(geometries):   
    a = []
    
    for location in geometries:
        lon = location.x
        lat = location.y

        url='http://matrix.mapzen.com/isochrone?json={"locations":[{"lat":'+str(lat)+',"lon":'+str(lon)+'}],"costing":"pedestrian","contours":[{"time":10}]}&api_key='+(MAPZEN_KEY)
        r = requests.get(url)
        
        a.append(Polygon(r.json()['features'][0]['geometry']['coordinates']))
        time.sleep(0.5)
        
    return a

In [91]:
beer_walksheds = generate_walksheds(beer.geometry)

In [94]:
coffee_walksheds = generate_walksheds(coffee.geometry)

In [113]:
beerWalk = gpd.GeoDataFrame()
beerWalk['geometry'] = beer_walksheds
beerWalk.crs = {'init' :'epsg:4326'}

In [114]:
coffeeWalk = gpd.GeoDataFrame()
coffeeWalk['geometry'] = coffee_walksheds
coffeeWalk.crs = {'init' :'epsg:4326'}

In [208]:
coffeeWalk.geometry.intersects(listing.geometry).value_counts()[True]

5

In [225]:
coffee_count = []

for geo in listings.geometry:
    counts = coffeeWalk.geometry.intersects(geo).value_counts()
    
    if True in counts.keys():
        coffee_count.append(counts[True])
    else:
        coffee_count.append(0)

In [237]:
listings['coffee_count'] = coffee_count

In [228]:
beer_count = []

for geo in listings.geometry:
    counts = beerWalk.geometry.intersects(geo).value_counts()
    
    if True in counts.keys():
        beer_count.append(counts[True])
    else:
        beer_count.append(0)

In [229]:
listings['beer_count'] = beer_count

In [240]:
listings.drop('coffeeshops', axis=1, inplace=True)

In [242]:
with open('NYClisting.geojson', 'w') as f:
   f.write(listings.to_json())